In [ ]:
from os.path import join
import jpype
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, isJVMStarted
import pandas as pd
import numpy as np
import re

In [ ]:
from os.path import join
import jpype
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, isJVMStarted
import pandas as pd
import numpy as np
import re

class Text_Cleaner:
    
    def __init__(self,):
        self.zemberek() 
              
    def text_cleaner(self,ds): #data_Seri - data.content
        # kullanmadan önce dropna yap
        array = []
        ds = ds.apply(lambda x: self.norm_doc(str(x)))
        ds = ds.str.replace("I","ı").str.replace("İ","i").str.lower().str.replace("[^\w\s]"," ")
        
        for i in ds: #kelime --> zemberek
            a =  ""
            for t in i.split():
                try:  
                    results: WordAnalysis = self.morphology.analyze(JString(t))
                        
                    if "Neg" in str(list(results)[0].formatLong()):
                        wrd = str(list(results)[0].getLemmas()[1])
                    else:
                        wrd = str(list(results)[0].getStems()[0])
                    a +=  wrd + " " 

                except:
                    continue
            a = re.sub(r'\b\w\b', "", a) # tekharfler silindi
            array.append(a)
            
        return array
        
        
    def norm_doc(self,sentence): 
        sentence = re.sub("'[A-Za-z]+", "", sentence) #tırnak eki silindi
        
        return re.sub("\d+", "", sentence) #sayılar silindi
    
    
    def zemberek(self,):
        ZEMBEREK_PATH: str = join('bin', 'zemberek-full.jar')
        if not jpype.isJVMStarted():   
            startJVM( getDefaultJVMPath(),'-ea',f'-Djava.class.path={ZEMBEREK_PATH}',convertStrings=False)
        TurkishMorphology: JClass = JClass('zemberek.morphology.TurkishMorphology') # burada java'da import etmişsin gibi düşün
        WordAnalysis: JClass = JClass('zemberek.morphology.analysis.WordAnalysis') # aynı şekilde
        self.morphology: TurkishMorphology = TurkishMorphology.createWithDefaults() # burada da o import'u kullanıyorsun
            
    def zemberek_shutdown(self,):
        shutdownJVM()
    

In [ ]:
data = pd.read_csv("ak.csv")
data = data[data['content'].notna()]

In [ ]:
tc = Text_Cleaner()

In [ ]:
data.content = tc.text_cleaner(data.content)

In [ ]:
data.content.head(10).tolist()

['BIST 100, 103 bini aşağı geçtiğinde hedge şart. Ancak 101 bin sağlam destek. Yine de 103 bin geçilmeden iyimserlik zor.',
 "BOEING'in üretiminde kusurlu olup olmadığı incelemeye başlanırken dünyadaki havayolu firmaları da bu model uçakların uçuşunu durdurdu. BIST ise bankacılık hisselerinin önderliğinde pozitif seyrini sürdürüyor.",
 'AKBNK, EREGL,THYAO, başta olmak üzere günün önemli şirket haberlerini Ziraat Yatırım analiz etti',
 'ALARK,EKGYO,GOLTS başta olmak üzere günün önemli şirket haberlerini Ziraat Yatırım analiz etti.',
 'Hazine ve Maliye Bakanlığı 10 milyon TL sermayeli İpotek Finansman Kurumu kuracak',
 'Bugün gerçekleşen TCMB, Para Politikası Kurulu toplantısından çıkan faizi sabit tutma kararı sonrası analistlerden yorumlar geldi.',
 "Norveç'in 1 trilyon dolarlık devlet varlık fonunun, 2018 sonu itibarıyla 707 milyon dolarını Türkiye'deki farklı şirketlerin hisse senetleriyle değerlendirdiği ortaya çıktı.\r\n",
 'İş Yatırım, Akbank için 7,85 TL hedef fiyat verdi',
 "Bor